In [5]:
import os
import requests
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.tools import StructuredTool
from langchain.agents import initialize_agent, AgentType
from langchain.tools import StructuredTool, Tool
from pydantic import BaseModel, Field
from langchain.tools import BaseTool
from typing import Any, Type
from langchain.utilities import DuckDuckGoSearchAPIWrapper
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.agents import create_sql_agent


llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash", 
    temperature=0.1,
    )

alpha_vantage_api_key = os.environ.get("ALPHA_VANTAGE_API_KEY")


In [6]:
db = SQLDatabase.from_uri("sqlite:///movies.sqlite")
toolkit = SQLDatabaseToolkit(db=db, llm=llm)


In [7]:
agent = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
)


In [9]:
agent.invoke(
    "Give me the movies that have the highest votes but the lowest budgets and give me the name of their directors also include their gross revenue. 한글로 답변해줘."
)




> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


directors, movies
Invoking: `sql_db_schema` with `{'table_names': 'movies, directors'}`



CREATE TABLE directors (
	name TEXT, 
	id INTEGER, 
	gender INTEGER, 
	uid INTEGER, 
	department TEXT, 
	PRIMARY KEY (id)
)

/*
3 rows from directors table:
name	id	gender	uid	department
James Cameron	4762	2	2710	Directing
Gore Verbinski	4763	2	1704	Directing
Sam Mendes	4764	2	39	Directing
*/


CREATE TABLE movies (
	id INTEGER, 
	original_title VARCHAR, 
	budget INTEGER, 
	popularity INTEGER, 
	release_date TEXT, 
	revenue INTEGER, 
	title TEXT, 
	vote_average REAL, 
	vote_count INTEGER, 
	overview TEXT, 
	tagline TEXT, 
	uid INTEGER, 
	director_id INTEGER DEFAULT 0 NOT NULL, 
	PRIMARY KEY (id)
)

/*
3 rows from movies table:
id	original_title	budget	popularity	release_date	revenue	title	vote_average	vote_count	overview	tagline	uid	director_id
43597	Avatar	237000000	150	2009-12-10	2787965087	Avatar	7.2	11800

{'input': 'Give me the movies that have the highest votes but the lowest budgets and give me the name of their directors also include their gross revenue. 한글로 답변해줘.',
 'output': '투표수가 가장 많고 예산이 가장 적은 영화 목록입니다. 감독 이름과 총 수익도 포함되어 있습니다.\n\n* **인셉션**: 감독 - 크리스토퍼 놀란, 수익 - 825,532,764 달러, 예산 - 1억 6천만 달러, 투표수 - 13752\n* **다크 나이트**: 감독 - 크리스토퍼 놀란, 수익 - 1,004,558,444 달러, 예산 - 1억 8천 5백만 달러, 투표수 - 12002\n* **아바타**: 감독 - 제임스 카메론, 수익 - 2,787,965,087 달러, 예산 - 2억 3천 7백만 달러, 투표수 - 11800\n* **어벤져스**: 감독 - 조스 웨던, 수익 - 1,519,557,910 달러, 예산 - 2억 2천만 달러, 투표수 - 11776\n* **데드풀**: 감독 - 팀 밀러, 수익 - 7억 8천 3백 1만 2천 9백 79달러, 예산 - 5천 8백만 달러, 투표수 - 10995\n* **인터스텔라**: 감독 - 크리스토퍼 놀란, 수익 - 6억 7천 5백 12만 1천 달러, 예산 - 1억 6천 5백만 달러, 투표수 - 10867\n* **장고: 분노의 추격자**: 감독 - 쿠엔틴 타란티노, 수익 - 4억 2천 5백 36만 8천 2백 38달러, 예산 - 1억 달러, 투표수 - 10099\n* **가디언즈 오브 갤럭시**: 감독 - 제임스 건, 수익 - 7억 7천 3백 32만 8천 6백 29달러, 예산 - 1억 7천만 달러, 투표수 - 9742\n* **헝거 게임**: 감독 - 게리 로스, 수익 - 6억 9천 1백 21만 692달러, 예산 - 7천 5백만 달러, 투표수 - 9455\n* **매드 맥스: 분노의 도로**: 감독 - 